In [1]:
# import team_utils as tu
import pandas as pd
# import numpy as np
# import requests
import my_utils as mu
from tqdm import tqdm
import json

In [63]:
def rawdataLoader(datas: str):
    return json.loads(datas.replace("'", '"').replace('True', '1').replace('False', '0'))

In [ ]:
def rawdataLoader1(datas: str):
    data_dict = json.loads(datas.replace("'", '"').replace('True', '1').replace('False', '0'))
    # 'teamPosition' 키가 없는 챔피언 정보에 대해서도 처리할 수 있도록 수정
    data_dict['champion']['teamPosition'] = data_dict['champion'].get('teamPosition', 'Unknown')
    return data_dict

### 챔피언/룬 (완료)

In [76]:
def champion_rune():
    
    df = mu.oracle_totalExecute("SELECT game_id, participant_number, champion, rune FROM RAWDATA")
    
    df = df.sort_values(['GAME_ID','PARTICIPANT_NUMBER'])
    df_cols = df.columns[2:]

    matches = {a:[] for a in df_cols}
    for rec in tqdm(df.iloc):
        for col in [a for a in df_cols]:
            matches[col].append(rawdataLoader(rec[col]))
    for col in [a for a in df_cols]:
        df[col] = matches[col]
    
    champion_list =  df['CHAMPION'].tolist()
    rune_list =  df['RUNE'].tolist()
    
    
    print('데이터 추출...')

    result = []
    col = ['championId','lane','runetype_core','core_mainrune','core_rune1','core_rune2','core_rune3','runetype_sub','sub_rune1','sub_rune2','shard_offense','shard_flex','shard_defence','win']
    
    for idx, champ in enumerate(tqdm(champion_list)):
        result.append([
            champ['championId'],
            champ['teamPosition'],
            rune_list[idx]['runePrimaryStyle'],
            rune_list[idx]['runeCorePerk'],
            rune_list[idx]['runePrimaryPerk1'],
            rune_list[idx]['runePrimaryPerk2'],
            rune_list[idx]['runePrimaryPerk3'],
            rune_list[idx]['runeSubStyle'],
            rune_list[idx]['runeSubPerk1'],
            rune_list[idx]['runeSubPerk2'],
            rune_list[idx]['runeShardOffense'],
            rune_list[idx]['runeShardFlex'],
            rune_list[idx]['runeShardDefense'],
            int(champ['win'])
        ])
    champ_rune = pd.DataFrame(result, columns=col)
     
    print('데이터 그룹화')
    champ_rune_count = champ_rune.groupby(['championId','lane','runetype_core','core_mainrune','runetype_sub']).count().rename(columns={'win':'games'})[['games']]
    champ_rune_win = champ_rune.groupby(['championId','lane','runetype_core','core_mainrune','runetype_sub']).sum()[['win']]
    
    result = champ_rune_count.join(champ_rune_win)
    result['win_rate'] = round(result.win / result.games * 100, 2)
    result.reset_index(inplace=True)
    
    return result

In [77]:
df = champion_rune()

oracle open!
oracle close!


1880it [00:00, 4823.45it/s]


데이터 추출...


100%|██████████| 1880/1880 [00:00<00:00, 376314.38it/s]

데이터 그룹화


In [78]:
df

,championId,lane,runetype_core,core_mainrune,runetype_sub,games,win,win_rate
0,1,MIDDLE,8100,8112,8200,1,1,100.00
1,1,MIDDLE,8100,8112,8300,1,1,100.00
2,1,MIDDLE,8200,8229,8300,1,0,0.00
3,1,MIDDLE,8300,8369,8200,3,1,33.33
4,1,UTILITY,8100,8112,8200,1,0,0.00
...,...,...,...,...,...,...,...,...
624,897,TOP,8400,8437,8000,1,0,0.00
625,897,TOP,8400,8437,8300,8,4,50.00
626,902,UTILITY,8200,8214,8300,7,4,57.14
627,902,UTILITY,8200,8214,8400,4,1,25.00


### 챔피언/스펠 (리팩터링 필요)

In [31]:
def champion_spell():
    
    df=mu.oracle_totalExecute("SELECT * FROM RAWDATA")
    
    print('파싱...')
    muts=[]
    for i in tqdm(range(len(df))):
        muts.append(rawdataLoader(df.loc[i]['MATCHES']))
    df['MATCHES'] = muts
    
    
    df = df.sort_values(['GAME_ID','PARTICIPANT_NUMBER'])
    
    matches_list =  df['MATCHES'].tolist()
    
    print('데이터 추출....')
    result=[]
    col = ['championId','lane','win','summoner1Id','summoner2Id']
    for i,matches in tqdm(enumerate(matches_list)):
        lst=[]
        lst.append(matches['champion']['championId'])
        lst.append(matches['champion']['teamPosition'])
        lst.append(matches['champion']['win'])
        lst.append(matches['spell']['summoner1Id'])
        lst.append(matches['spell']['summoner2Id'])
    
        result.append(lst)
    champ_spell = pd.DataFrame(result,columns=col)
    
    
    champ_spell_win = champ_spell.groupby(['championId','lane','summoner1Id','summoner2Id']).sum() #서머너스펠 챔피언 승리한 갯수
    champ_spell_count = champ_spell.groupby(['championId','lane','summoner1Id','summoner2Id']).count().rename(columns={'win':'games'}) #서머너스펠 챔피언 픽한 갯수

    
    result = champ_spell_count.join(champ_spell_win)
    
    print(result)
    
    result['win_rate']= round(result.win/result.games * 100,2)
    result['pick_rate']=round(result.games/(sum(result.games)/10)*100,2)
    result.reset_index(inplace=True)
    
    return result

In [32]:
df2 = champion_spell()

oracle open!
oracle close!
파싱...


100%|████████████████████████████████████████████████████████████████████████| 141440/141440 [00:34<00:00, 4092.72it/s]


데이터 추출....


141440it [00:04, 31377.93it/s]


                                            games  win
championId lane    summoner1Id summoner2Id            
1                  14          4                1    0
           BOTTOM  14          4                1    0
           MIDDLE  1           4                2    0
                   3           4                1    1
                   4           1                1    1
...                                           ...  ...
902        UTILITY 4           21               1    1
                   7           4              597  311
                               6                1    0
                   14          4              169   88
                   21          4                2    2

[3276 rows x 2 columns]


In [17]:
df2

,championId,lane,summoner1Id,summoner2Id,games,win,win_rate,pick_rate
0,1,,14,4,1,0,0.00,0.01
1,1,BOTTOM,14,4,1,0,0.00,0.01
2,1,MIDDLE,1,4,2,0,0.00,0.01
3,1,MIDDLE,3,4,1,1,100.00,0.01
4,1,MIDDLE,4,1,1,1,100.00,0.01
...,...,...,...,...,...,...,...,...
3271,902,UTILITY,4,21,1,1,100.00,0.01
3272,902,UTILITY,7,4,597,311,52.09,4.22
3273,902,UTILITY,7,6,1,0,0.00,0.01
3274,902,UTILITY,14,4,169,88,52.07,1.19


In [7]:
df2[df2['championId']==1].iloc[0]

championId        1
lane               
summoner1Id      14
summoner2Id       4
games             1
win               0
win_rate        0.0
pick_rate      0.01
Name: 0, dtype: object

### 챔피언/초기 정보 (리팩터링 필요)

In [25]:
def get_champion_info():
    
    rawdata = mu.oracle_totalExecute("SELECT * FROM RAWDATA WHERE VERSION='13.9.506.4846'") #데이터 불러오기
    
    print('파싱시작했다~')
    muts=[]
    for i in tqdm(range(len(rawdata))):
        muts.append(rawdataLoader(rawdata.loc[i]['MATCHES']))
    rawdata['MATCHES'] = muts #데이터 파싱
    print('파싱끝났다~')
    
    df = pd.DataFrame(rawdata) #df화
    
    df = df.sort_values(['GAME_ID','PARTICIPANT_NUMBER']) #정렬
    
    matches_list = df['MATCHES'].tolist()
    result = []
    col = ['championId','lane','win','bans']
    for i, matches in tqdm(enumerate(matches_list)):
        lst = []
        lst1 = []
        lst.append(matches['champion']['championId']) # 챔피언 이름
        lst.append(matches['champion']['teamPosition']) # 챔피언이 간 라인
        lst.append(matches['champion']['win']) #승패
        for j in range(len(matches['game']['bans'])):
            lst1.append(matches['game']['bans'][j]['championId']) #밴 1~10까지 픽턴에따른
        lst.append(lst1)    
        result.append(lst)
    champion_info_data = pd.DataFrame(result, columns = col)
    print("데이터 정제 삽입 완료!")
        
    # ======================================================================================================================
        
#     champion_info_data2 = champion_info_data[['championId','lane','win']] # 그 외의 것
#     champion_info_data2 = champion_info_data.astype({'win':'int'}) # bool -> int
    champion_info_data2 = champion_info_data[['championId','lane','win']] # 그 외의 것
    champion_info_data2 = champion_info_data2.astype({'win':'int', 'lane':'str'}) # bool -> int, object -> str


    champion_info_pick= champion_info_data2.groupby(['championId', 'lane']).count().rename(columns={'win':'playgame'}) #픽된 수
    champion_info_pick['total_game'] = champion_info_pick['playgame'].sum()/10 # 총 게임

    champion_info_pick['picks_rate']= round((champion_info_pick.playgame/champion_info_pick.total_game)*100,3) # 픽률 뽑기

    champion_info_sum= champion_info_data2.groupby(['championId', 'lane']).sum() # 승리 횟수 1인거 더하기

    champion_info_wins_rate = champion_info_sum.join(champion_info_pick) # 승리, 게임수 
    champion_info_wins_rate['wins_rate'] = round((champion_info_wins_rate.win/champion_info_wins_rate.playgame)*100,3) # 승률 뽑기
    
    print(champion_info_wins_rate)

    # ======================================================================================================================
    # ====================================================== ban_rate ======================================================
    # ======================================================================================================================
    
    rawdata.sort_values(['GAME_ID', 'PARTICIPANT_NUMBER'], ascending=[False, True], ignore_index=True, inplace=True)

    ban_list = []
    for loc in range(0, len(rawdata), 10):
        matches = rawdata.iloc[loc]['MATCHES']
        bans = [a['championId'] for a in matches['game']['bans']]
        ban_list.append({'game_id':rawdata.iloc[loc]['GAME_ID'], 'bans':bans})

    champs = mu.oracle_totalExecute('SELECT CHAMP_ID FROM CHAMPDATA')

    champ_id_list = [a for a in champs['CHAMP_ID']]

    ban_time = {a:0 for a in champ_id_list}

    for lst in ban_list:
        for ban in lst['bans']:
            if ban == -1: continue
            ban_time[ban] += 1

    ban_rate = {a:round((ban_time[a] / len(ban_list) * 100), 2) for a in ban_time}

    ban_rate2 = pd.DataFrame([ban_rate]).T ## 정렬 및 데이터프레임화

    ban_rate3 = ban_rate2.reset_index()
    ban_rate3.columns=['championId','ban_rate']
    

    champion_info_put = pd.merge(champion_info_wins_rate,ban_rate3,on=['championId'])
    
    champion_info_put
    return champion_info_data,champion_info_put

In [28]:
df4,df3 = get_champion_info()

oracle open!
oracle close!
파싱시작했다~


100%|██████████████████████████████████████████████████████████████████████████| 31100/31100 [00:06<00:00, 4664.10it/s]


파싱끝났다~


31100it [00:00, 170048.96it/s]


데이터 정제 삽입 완료!
                    win  playgame  total_game  picks_rate  wins_rate
championId lane                                                     
1          MIDDLE    42        85      3110.0       2.733     49.412
           TOP        0         2      3110.0       0.064      0.000
           UTILITY   15        34      3110.0       1.093     44.118
2          BOTTOM     1         2      3110.0       0.064     50.000
           JUNGLE     1         3      3110.0       0.096     33.333
...                 ...       ...         ...         ...        ...
895        BOTTOM    24        39      3110.0       1.254     61.538
897        MIDDLE    26        57      3110.0       1.833     45.614
           TOP      141       272      3110.0       8.746     51.838
           UTILITY    2         4      3110.0       0.129     50.000
902        UTILITY  186       354      3110.0      11.383     52.542

[513 rows x 5 columns]
oracle open!
oracle close!


In [30]:
df4[df4['lane']=='']

,championId,lane,win,bans
3230,54,,0,"[267, 711, 121, 166, 122, 555, 154, 238, 126, 25]"
6150,41,,0,"[126, 39, 53, 166, 60, 29, 126, 69, 53, 121]"
24462,84,,0,"[7, 15, 121, 114, 30, 78, 497, 121, 114, 902]"
